In [ ]:
import csv
import numpy as np
import utils_deck_generation as idg
import healpy_pointings as hpoint
import netcdf_read_write as nrw

In [ ]:
imap_nside = 256
imap_npix = imap_nside**2 * 12

# pointings
filename_pointing = "pointing_index_"
surface_cover_radians = np.radians(20.0)
pointing_nside = 3

# defocus
filename_defocus = "defocus_index_"
defocus_range = 10.0 # mm
num_defocus = 3

#power
power_range = 1.0 # fraction of full power
num_powers = 1

run_sims = True
run_compression = True
run_type = "nif" # "test"
num_quads = 24

## Import NIF: Beam, Quad and Cone data

In [ ]:
the_data = dict()

the_data['nbeams'] = 4
the_data['target_radius'] = 1100.0
the_data['facility'] = "NIF"
the_data['nbeams_total'] = 192

In [ ]:
filename = "NIF_UpperBeams.txt"
j = -1

f=open(filename, "r")
reader = csv.reader(f, delimiter='\t')
for row in reader:
    if j==-1:
        key = row
        for i in range(len(row)):
            the_data[row[i]] = [None] * int(the_data['nbeams_total'])
    else:
        for i in range(len(row)):
            if i < 2:
                the_data[key[i]][j] = row[i]
            elif i < 5:
                the_data[key[i]][j] = float(row[i])
            else:
                the_data[key[i]][j] = int(row[i])
    j=j+1
f.close()
filename = "NIF_LowerBeams.txt"
f=open(filename, "r")
reader = csv.reader(f, delimiter='\t')
for row in reader:
    if j>96:
        for i in range(len(row)):
            if i < 2:
                the_data[key[i]][j-1] = row[i]
            elif i < 5:
                the_data[key[i]][j-1] = float(row[i])
            else:
                the_data[key[i]][j-1] = int(row[i])
    j=j+1
f.close()

the_data["Theta"] = np.radians(the_data["Theta"])
the_data["Phi"] = np.radians(the_data["Phi"])

In [ ]:
quad_from_each_cone = ('Q15T', 'Q13T', 'Q14T', 'Q11T')

## Run Quad and Rotate to form Cone

In [ ]:
the_data["p0"] = [1./4.] * the_data["nbeams"]
the_data["fuse"] = [False] * the_data["nbeams"]

defocus_array = np.linspace(0.0, defocus_range, num_defocus)

for quad_name in quad_from_each_cone:
    quad_slice = slice(the_data["Quad"].index(quad_name),the_data["Quad"].index(quad_name)+4)
    beam_names = the_data['Beam'][quad_slice]

    ## Create Healpy grid of pointings
    x, y, z, theta_offset = hpoint.theta_pointing_rotations(the_data, quad_slice, pointing_nside, surface_cover_radians)
    r = np.sqrt(x**2 + y**2 + z**2)
    pointing_theta = np.arccos(z/r)
    pointing_phi = np.arctan2(y,x)
    
    for dind in range(len(defocus_array)):
        the_data["defocus"] = [defocus_array[dind]] * the_data["nbeams"]
        for pind in range(len(theta_offset)):
            ## pointing data
            pointings=dict()
            for __name in beam_names:
                __x = float(x[pind])
                __y = float(y[pind])
                __z = float(z[pind])
                pointings[__name]=np.array((__x,__y,__z))
            the_data['pointings'] = []
            for beam in beam_names:
                for key, value in pointings.items():
                    if (key == beam):
                        the_data['pointings'].append(np.asarray(value))

            run_location = filename_pointing + str(pind) + "_" + filename_defocus + str(dind)
            idg.generate_input_deck(the_data, run_location)
            idg.generate_input_pointing_and_pulses(the_data, run_location, quad_slice, run_type)

            if run_sims:
                b='Running ifriit in: '+run_location+"  "
                print("\r", b, end="")
                !cd $run_location;chmod +x main;mpirun -np 1 ./main > a.out

            ## Compression of cone
            if run_compression:
                hpxmap, n_beams = nrw.read_intensity(run_location, run_type, beam_names, imap_nside)
                nrw.rotate_cone_and_save(the_data, quad_name, hpxmap, imap_nside, pointing_theta[pind], pointing_phi[pind], run_location)

## Diagnostic

In [ ]:
import matplotlib.pyplot as plt
import healpy as hp
import netcdf_read_write as nrw

In [ ]:
pind = 2
dind = 0
run_location = filename_pointing + str(pind) + "_" + filename_defocus + str(dind)

quad_name = 'Q15T'
quad_slice = slice(the_data["Quad"].index(quad_name),the_data["Quad"].index(quad_name)+4)
beam_names = the_data['Beam'][quad_slice]

hpxmap, n_beams = nrw.read_intensity(run_location, run_type, beam_names, imap_nside)

r = the_data['target_radius'] / 1.0e4 # microns to cm
total_TW = np.mean(hpxmap)*10**(-12)
surface_area = 4.0*np.pi*r**2

#rms
avg_power = np.mean(hpxmap)
hpxmap_rms = 100.0 * np.sqrt(np.mean((hpxmap / avg_power - 1.0)**2))
hpxmap_rms_spatial = 100.0 * np.sqrt((hpxmap / avg_power - 1.0)**2)

print('')
print('The total power deposited is ', total_TW*surface_area, 'TW')
print('The power per beam deposited is ', total_TW*surface_area/n_beams, 'TW')
print('This is a drive efficiency of ', total_TW*surface_area/(n_beams*0.25)*100, '%')
print('Number of beams is ', n_beams)
#print('RMS is ', hpxmap_rms, '%')
print('')

hp.mollview(hpxmap,unit=r"$\rm{W/cm^2}$",flip="geo")
hp.graticule()

port_theta = np.mean(the_data['Theta'][quad_slice])
port_phi = np.mean(the_data['Phi'][quad_slice])
hp.projscatter(port_theta, port_phi)
quad_pointing_theta = port_theta + theta_offset[pind]
quad_pointing_phi = port_phi
hp.projscatter(quad_pointing_theta, quad_pointing_phi)

## Check cone compression

In [ ]:
from netCDF4 import Dataset
import numpy as np
import healpy as hp
import matplotlib.pyplot as plt

In [ ]:
power_fractions = np.linspace(power_range, 1, num_powers)

In [ ]:
intensity_map = 0.0
pointing_per_cone = [0,0,0,0]
defocus_per_cone = [0,0,0,0]
power_per_cone = [0,0,0,0]
figure_location = "plots"

portloc_theta = np.zeros(num_quads)
portloc_phi = np.zeros(num_quads)
quad_pointing_theta = np.zeros(num_quads)
quad_pointing_phi = np.zeros(num_quads)

iquad = 0
icone = 0

for quad_name in quad_from_each_cone:
    quad_slice = slice(the_data["Quad"].index(quad_name),the_data["Quad"].index(quad_name)+4)

    cone_name = the_data['Cone'][quad_slice]
    cone_name = cone_name[0]
    quad_name = the_data['Quad'][quad_slice]
    quad_name = quad_name[0]

    beam_count = int(the_data["Cone"].count(cone_name)/2)
    cone_slice = slice(the_data["Cone"].index(cone_name),the_data["Quad"].index(quad_name)+beam_count,4)
    quad_list_in_cone = the_data["Quad"][cone_slice]

    cone_name = str(cone_name)
    cone_name = cone_name.replace(".", "_")
    run_location = filename_pointing + str(pointing_per_cone[icone]) + "_" + filename_defocus + str(defocus_per_cone[icone])
    save_name = run_location + "/" + run_location + "_cone_" + cone_name + ".nc"

    for i in range(int(beam_count/4)):
        cone_data = Dataset(save_name)
        intensity_data = cone_data[quad_list_in_cone[i]]
        port_loc = intensity_data.variables["port_loc"][:]
        portloc_theta[iquad] = port_loc[1]
        portloc_phi[iquad] = port_loc[2]
        quad_pointing = intensity_data.variables["quad_pointing"][:]
        quad_pointing_theta[iquad] = quad_pointing[1]
        quad_pointing_phi[iquad] = quad_pointing[2]
        power_mult = power_fractions[power_per_cone[icone]]
        intensity_map = intensity_map + intensity_data.variables["intensity_map"][:] * power_mult

        iquad = iquad + 1
    icone = icone + 1

intensity_map = intensity_map + np.flip(intensity_map)

hp.mollview(intensity_map, unit=r"$\rm{W/cm^2}$", flip="geo")
hp.graticule()

hp.projscatter(portloc_theta, portloc_phi)
hp.projscatter(quad_pointing_theta, quad_pointing_phi)

quad_name = "Q45B"
quad_slice = slice(the_data["Quad"].index(quad_name),the_data["Quad"].index(quad_name)+4)
bottom_portloc_theta = np.mean(the_data['Theta'][quad_slice])
bottom_portloc_phi = np.mean(the_data['Phi'][quad_slice])
hp.projscatter(bottom_portloc_theta, bottom_portloc_phi)
plt.savefig(figure_location+'/intensity_mollweide.png', dpi=300, bbox_inches='tight')

## Incident fraction, RMS and modes

In [ ]:
n_beams = the_data['nbeams_total']
r = the_data['target_radius'] / 1.0e4 # microns to cm
total_TW = np.mean(intensity_map)*10**(-12)
surface_area = 4.0 * np.pi * r**2

#rms
avg_power = np.mean(intensity_map)
intensity_map_rms = 100.0 * np.sqrt(np.mean((intensity_map / avg_power - 1.0)**2))
intensity_map_rms_spatial = 100.0 * np.sqrt((intensity_map / avg_power - 1.0)**2)

print('')
print('The total power deposited is ', total_TW * surface_area, 'TW')
print('The power per beam deposited is ', total_TW * surface_area / n_beams, 'TW')
print('This is a drive efficiency of ', total_TW * surface_area / (n_beams * 0.25) * 100.0, '%')
print('RMS is ', intensity_map_rms, '%')
print('Number of beams ', n_beams)
print('')

In [ ]:
hp.mollview(intensity_map_rms_spatial,unit="RMS",flip="geo")
plt.savefig(figure_location+'/rms_mollweide.png', dpi=300, bbox_inches='tight')
hp.graticule()

In [ ]:
intensity_map_normalized = 100.0 * (intensity_map / avg_power - 1.0)

# Compute the corresponding normalized mode spectrum
LMAX = 100
rmsalms = hp.sphtfunc.map2alm(intensity_map_normalized, lmax=LMAX)
var = abs(rmsalms)**2
the_modes = np.zeros(LMAX)
power_spectrum = np.zeros(LMAX)
for l in range(LMAX):
    for m in range(l):
        if (m>0):
            the_modes[l] = the_modes[l] + 2.*var[hp.sphtfunc.Alm.getidx(LMAX, l, m)]
        else:          
            the_modes[l] = the_modes[l] + var[hp.sphtfunc.Alm.getidx(LMAX, l, m)]
    power_spectrum[l] = (2.0 * l + 1.0) * the_modes[l] / (4.0 * np.pi)

power_spectrum_unweighted = np.sqrt(the_modes)
power_spectrum_weighted = np.sqrt(power_spectrum)

x_max = 20

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), power_spectrum_unweighted)
ax.set_xticks(range(0,100))
plt.xlim([0, x_max])
plt.title("Unweighted Modes LLE")
plt.xlabel("l mode")
plt.ylabel(r"rms amplitude ($\%$)")
plt.savefig(figure_location+"/unweighted_modes.png", dpi=300, bbox_inches='tight')
print("The LLE quoted rms cumalitive over all modes is: ", np.sqrt(np.sum(the_modes**2)), "%")

fig = plt.figure()
ax = plt.axes()
plt.plot(np.arange(LMAX), power_spectrum_weighted)
ax.set_xticks(range(0,100))
plt.xlim([0, x_max])
plt.title("Weighted Modes")
plt.xlabel("l mode")
plt.ylabel(r"rms amplitude ($\%$)")
plt.savefig(figure_location+"/weighted_modes.png", dpi=300, bbox_inches='tight')